In [1]:
#GA.xlsx & PSO.xlsx

In [2]:
import os
import pandas as pd
import numpy as np
from openpyxl.styles import Alignment, Font, PatternFill
from openpyxl.utils import get_column_letter
import xlwings as xw

# Function to get all directories starting with 'GA' or 'PSO'
def get_ga_directories(parent_folder):
    return [d for d in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, d)) and (d.startswith('GA') or d.startswith('PSO'))]

# Set the parent folder where all GA or PSO directories are located
parent_folder = '.'  # Change this to the actual path if needed

# Get the list of top-level GA or PSO directories
ga_directories = get_ga_directories(parent_folder)

def try_convert_numeric(val):
    try:
        return pd.to_numeric(val)
    except ValueError:
        return val   

def save_and_calculate_excel(file_path):
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    #print(f"Attempting to open: {file_path}")
    
    app = xw.App(visible=False)
    workbook = None  # Initialize workbook to None

    try:
        workbook = app.books.open(file_path)
        workbook.app.calculate()
        workbook.save()
    except Exception as e:
        print(f"Error with the Excel file {file_path}: {e}")
    finally:
        if workbook is not None:
            workbook.close()
        app.quit()
    
# Process each GA directory
for ga_dir in ga_directories:
    ga_path = os.path.join(parent_folder, ga_dir)
    excel_file = os.path.join(parent_folder, f'{ga_dir}.xlsx')
    
    # Get all subdirectories in the GA directory
    subdirectories = [d for d in os.listdir(ga_path) if os.path.isdir(os.path.join(ga_path, d)) and d.startswith(ga_dir)]
    
    # Initialize an Excel writer for this GA
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        # Process each subdirectory (experiment)
        for subdir in subdirectories:
            combined_df = pd.DataFrame()
            
            # Loop over each file in the subdirectory
            for i in range(1, 11):
                file_name = os.path.join(ga_path, subdir, f'{subdir}-Result{i}.txt')
                column_name = f'{i}'
                
                if os.path.isfile(file_name):
                    try:
                        with open(file_name, 'r') as file:
                            content = file.read().splitlines()
                        
                        # Create a DataFrame and attempt to convert to numeric
                        df = pd.DataFrame({column_name: content})
                        df[column_name] = df[column_name].apply(try_convert_numeric)
                        
                        combined_df = pd.concat([combined_df, df], axis=1)
                    except Exception as e:
                        print(f"Error reading file {file_name}: {e}")
                        continue
                else:
                    print(f"File not found: {file_name}")
                    continue
            
            if not combined_df.empty:
                try:
                    # Ensure numeric columns are written as numbers
                    combined_df = combined_df.apply(pd.to_numeric)
                    
                    # Create the 'Generation \ Experiment' column
                    gen_exp_column = list(range(1, 2001))  # First 2000 rows
                    gen_exp_column.append('')  # Skip 1 line
                    gen_exp_column.append('Values')  # Add 'Values'
                    gen_exp_column.extend(range(1, 31))  # Add 1 to 30
                    gen_exp_column.append('')  # Skip 1 line
                    gen_exp_column.append('Time')  # Add 'Time'
                    gen_exp_column.append('')  # Skip 1 line
                    
                    # Create a new DataFrame with the correct number of rows
                    new_df = pd.DataFrame(index=range(len(gen_exp_column)))
                    
                    # Add the 'Generation \ Experiment' column
                    new_df['Generation \\ Experiment'] = gen_exp_column
                    
                    # Add data from combined_df, filling with empty strings where necessary
                    for col in combined_df.columns:
                        new_df[col] = combined_df[col].tolist() + [''] * (len(new_df) - len(combined_df))
                    
                    # Write to Excel
                    new_df.to_excel(writer, sheet_name=subdir, index=False)
                    
                    # Get the worksheet
                    worksheet = writer.sheets[subdir]
                    
                    worksheet.insert_rows(2002, 2)
                    
                    # Average Fitness
                    # Min and Max rows
                    worksheet.cell(row=2002, column=1, value='Min')
                    worksheet.cell(row=2002, column=2, value='=MIN(B2:B2001)')
                    worksheet.cell(row=2002, column=3, value='=MIN(C2:C2001)')
                    worksheet.cell(row=2002, column=4, value='=MIN(D2:D2001)')
                    worksheet.cell(row=2002, column=5, value='=MIN(E2:E2001)')
                    worksheet.cell(row=2002, column=6, value='=MIN(F2:F2001)')
                    worksheet.cell(row=2002, column=7, value='=MIN(G2:G2001)')
                    worksheet.cell(row=2002, column=8, value='=MIN(G2:G2001)')
                    worksheet.cell(row=2002, column=9, value='=MIN(H2:H2001)')
                    worksheet.cell(row=2002, column=10, value='=MIN(I2:I2001)')
                    worksheet.cell(row=2002, column=11, value='=MIN(J2:J2001)')
                    
                    worksheet.cell(row=2003, column=1, value='Max')
                    worksheet.cell(row=2003, column=2, value='=MAX(B2:B2001)')
                    worksheet.cell(row=2003, column=3, value='=MAX(C2:C2001)')
                    worksheet.cell(row=2003, column=4, value='=MAX(D2:D2001)')
                    worksheet.cell(row=2003, column=5, value='=MAX(E2:E2001)')
                    worksheet.cell(row=2003, column=6, value='=MAX(F2:F2001)')
                    worksheet.cell(row=2003, column=7, value='=MAX(G2:G2001)')
                    worksheet.cell(row=2003, column=8, value='=MAX(G2:G2001)')
                    worksheet.cell(row=2003, column=9, value='=MAX(H2:H2001)')
                    worksheet.cell(row=2003, column=10, value='=MAX(I2:I2001)')
                    worksheet.cell(row=2003, column=11, value='=MAX(J2:J2001)')
                    
                    # Add headers for min, max, and average
                    worksheet.cell(row=2001, column=12, value='Min Lowest')
                    worksheet.cell(row=2001, column=13, value='Max Lowest')
                    worksheet.cell(row=2001, column=14, value='Average Fitness')
                    
                    worksheet.cell(row=2002, column=12, value='=MIN(B2002:K2002)')
                    worksheet.cell(row=2002, column=13, value='=MAX(B2002:K2002)')
                    worksheet.cell(row=2002, column=14, value='=AVERAGE(B2002:K2002)')
                    
                    # Average Time                   
                    # Add headers for min, max, and average
                    worksheet.cell(row=2036, column=12, value='Min Time')
                    worksheet.cell(row=2036, column=13, value='Max Time')
                    worksheet.cell(row=2036, column=14, value='Average Time')

                    worksheet.cell(row=2037, column=12, value='=MIN(B2037:K2037)')
                    worksheet.cell(row=2037, column=13, value='=MAX(B2037:K2037)')
                    worksheet.cell(row=2037, column=14, value='=AVERAGE(B2037:K2037)')
                    
                    # Adjust column widths and alignment
                    for col in range(1, 15):  # Columns A to N
                        max_len = max(len(str(cell.value)) for cell in worksheet[get_column_letter(col)])
                        worksheet.column_dimensions[get_column_letter(col)].width = max_len + 2
                        
                        # Center align all columns except the first one
                        if col > 1:
                            for cell in worksheet[get_column_letter(col)]:
                                cell.alignment = Alignment(horizontal='center')
                    
                    # Highlight and bold Average Fitness
                    worksheet.cell(row=2001, column=14).fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
                    worksheet.cell(row=2036, column=14).fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
                    
                    worksheet.cell(row=2001, column=14).font = Font(bold=True)
                    worksheet.cell(row=2036, column=14).font = Font(bold=True)
                    
                    # Numbering format
                    for row in range(2, 2004):
                        for col in range(2, 12):
                            worksheet.cell(row=row, column=col).number_format = '0.000000E+00'
                    
                    for row in range(2006, 2036):
                        for col in range(2, 12):
                            worksheet.cell(row=row, column=col).number_format = '0.000000'
                    
                    for col in range(2, 15):
                            worksheet.cell(row=2037, column=col).number_format = '0.000000'
                    
                    worksheet.cell(row=2002, column=12).number_format ='0.000000E+00'
                    worksheet.cell(row=2002, column=13).number_format ='0.000000E+00'
                    worksheet.cell(row=2002, column=14).number_format ='0.000000E+00'
                    
                except Exception as e:
                    print(f"Error writing to Excel sheet {subdir}: {e}")
                
            else:
                print(f"No files found for directory: {subdir}")
            
    save_and_calculate_excel(excel_file)
            
    print(f"Created Excel file: {excel_file}")

print("Done processing all GA / PSO directories.")

Created Excel file: .\GA1.xlsx
Created Excel file: .\GA10.xlsx
Created Excel file: .\GA100.xlsx
Created Excel file: .\GA101.xlsx
Created Excel file: .\GA102.xlsx
Created Excel file: .\GA103.xlsx
Created Excel file: .\GA104.xlsx
Created Excel file: .\GA105.xlsx
Created Excel file: .\GA106.xlsx
Created Excel file: .\GA107.xlsx
Created Excel file: .\GA108.xlsx
Created Excel file: .\GA109.xlsx
Created Excel file: .\GA11.xlsx
Created Excel file: .\GA110.xlsx
Created Excel file: .\GA111.xlsx
Created Excel file: .\GA112.xlsx
Created Excel file: .\GA113.xlsx
Created Excel file: .\GA114.xlsx
Created Excel file: .\GA115.xlsx
Created Excel file: .\GA116.xlsx
Created Excel file: .\GA117.xlsx
Created Excel file: .\GA118.xlsx
Created Excel file: .\GA119.xlsx
Created Excel file: .\GA12.xlsx
Created Excel file: .\GA120.xlsx
Created Excel file: .\GA121.xlsx
Created Excel file: .\GA122.xlsx
Created Excel file: .\GA123.xlsx
Created Excel file: .\GA124.xlsx
Created Excel file: .\GA125.xlsx
Created Excel f

Created Excel file: .\GA96.xlsx
Created Excel file: .\GA97.xlsx
Created Excel file: .\GA98.xlsx
Created Excel file: .\GA99.xlsx
Done processing all GA / PSO directories.


In [3]:
#P04G01.xlsx

In [2]:
import os
import re
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle

# Define source and destination paths
source_folder = '.'  # Change this to the path of your source folder
destination_file = 'P04G01.xlsx'  # Change this to your destination file

# Define cells to copy from and their corresponding destination columns
source_cells = ['N2002', 'N2037']  # Cells to copy from each sheet
destination_columns = ['D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']  # Columns to write the data into

# Define starting rows
start_row_for_n2002 = 7  # Starting row for N2002 data
start_row_for_n2037 = 269  # Starting row for N2037 data

# Define starting columns
start_col_idx = 4  # Column 'D' corresponds to index 4

# Create a style for scientific number formatting
scientific_style = NamedStyle(name='scientific_style')
scientific_style.number_format = '0.000000E+00'  # Format to scientific notation with 6 decimal places

# Load the existing destination workbook
dest_wb = load_workbook(destination_file)
dest_sheet = dest_wb.active  # This gets the first sheet in the workbook

# Function to extract numerical part from filenames
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(0)) if match else float('inf')

# List and sort the filenames numerically
files = [filename for filename in os.listdir(source_folder) if filename.startswith('GA') and filename.endswith('.xlsx')]
files.sort(key=extract_number)

# Process each source file
for filename in files:
    file_path = os.path.join(source_folder, filename)
    wb = load_workbook(file_path, data_only=True)
    
    # Collect data from each source cell
    collected_values = {cell: [] for cell in source_cells}
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
        for cell in source_cells:
            cell_value = sheet[cell].value
            collected_values[cell].append(cell_value)
    
    # Write collected values to the destination sheet
    for cell in source_cells:
        if cell == 'N2002':
            start_row = start_row_for_n2002
            start_row_for_n2002 += (len(collected_values[cell]) // len(destination_columns)) + (1 if len(collected_values[cell]) % len(destination_columns) != 0 else 0)
        elif cell == 'N2037':
            start_row = start_row_for_n2037
            start_row_for_n2037 += (len(collected_values[cell]) // len(destination_columns)) + (1 if len(collected_values[cell]) % len(destination_columns) != 0 else 0)
        else:
            continue  # Skip if the cell is not in the specified list
        
        # Write each cell's data to the destination sheet with 10 values per row
        row_data = collected_values[cell]
        for i in range(0, len(row_data), len(destination_columns)):
            data_chunk = row_data[i:i + len(destination_columns)]  # Slice the data into chunks of 10
            for col_idx, value in enumerate(data_chunk):
                dest_cell = dest_sheet.cell(row=start_row, column=start_col_idx + col_idx)  # Write to the correct row and column
                dest_cell.value = value
                # Apply scientific number formatting
                #dest_cell.style = scientific_style
            start_row += 1  # Move to the next row after writing a chunk

# Save the destination workbook
dest_wb.save(destination_file)

print("Data copied successfully!")


Data copied successfully!
